In [1]:
pip install fake-useragent

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [54]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from fake_useragent import UserAgent  # Install this library using: pip install fake-useragent

def scrape_amazon(search_keyword, num_pages):
    base_url = 'https://www.amazon.in/s'
    data = []

    headers = {'User-Agent': UserAgent().random}

    for page in range(1, num_pages + 1):
        params = {'k': search_keyword, 'page': page}
        
        try:
            webpage = requests.get(base_url, params=params, headers=headers)
            webpage.raise_for_status()  # Raise an HTTPError for bad responses
            
            soup = BeautifulSoup(webpage.content, 'html.parser')

            brands = soup.find_all('div', class_='puis-card-container s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis puis-v1x9xtdrqwkf2b2sy3gv3bhvhpn s-latency-cf-section puis-card-border')

            for brand in brands:
                name = brand.find('h2').text.strip()
                rating = brand.find('span', class_='a-icon-alt').text.strip() if brand.find('span', class_='a-icon-alt') else 'N/A'
                reviews = brand.find('span', class_='a-size-base s-underline-text').text.strip() if brand.find('span', class_='a-size-base s-underline-text') else 'N/A'
                price = brand.find('span', class_='a-price-whole').text.strip() if brand.find('span', class_='a-price-whole') else 'N/A'

                data.append({'brand_name': name, 'rating': rating, 'reviews': reviews, 'price': price})

            time.sleep(2)  # Add a delay of 2 seconds between requests to avoid overloading the server
            
        except requests.exceptions.HTTPError as errh:
            print ("HTTP Error:", errh)
        except requests.exceptions.ConnectionError as errc:
            print ("Error Connecting:", errc)
        except requests.exceptions.Timeout as errt:
            print ("Timeout Error:", errt)
        except requests.exceptions.RequestException as err:
            print ("Something went wrong:", err)

    return pd.DataFrame(data)

# Example usage
search_keyword = 'Tv'
num_pages_to_scrape = 3
df = scrape_amazon(search_keyword, num_pages_to_scrape)
df

HTTP Error: 503 Server Error:  for url: https://www.amazon.in/s?k=Tv&page=2
HTTP Error: 503 Server Error:  for url: https://www.amazon.in/s?k=Tv&page=3


,brand_name,rating,reviews,price
0,iFFALCON,4.1 out of 5 stars,"3,562","7,990"
1,Samsung,4.2 out of 5 stars,"12,955","14,990"
2,Redmi,4.2 out of 5 stars,"59,956","13,999"
3,MI,4.2 out of 5 stars,"55,897","14,990"
4,LG,N/A,N/A,"24,999"
5,iFFALCON,4.1 out of 5 stars,"3,562","24,999"
6,Acer,4.3 out of 5 stars,"14,123","10,999"
7,iFFALCON,4.1 out of 5 stars,"3,562","17,999"
8,LG,4.3 out of 5 stars,"14,762","14,990"
9,Acer,4.1 out of 5 stars,"5,189","12,999"
